In [ ]:
!pip install langchain
!pip install chromadb
#!pip install sentence_transformers #necessary for Huggingface Embeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain import LLMChain
from google.colab import drive
import os
import json
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

In [ ]:
# We need to add the OpenAI API key to the environment variables for using embeddings and llm.
os.environ["OPENAI_API_KEY"] = "YOUR API KEY HERE"

In [ ]:
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
Working_Dir = os.path.join(MOUNTPOINT, 'My Drive', 'London Knowledge Prompting Hackathon')
drive.mount(MOUNTPOINT)
print(Working_Dir)

Mounted at /content/gdrive
/content/gdrive/My Drive/London Knowledge Prompting Hackathon


In [ ]:
# Read jsonl file containing LM-KBC data
def read_lm_kbc_jsonl(file_path: str):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [ ]:
file_path = Working_Dir + '/data/train.jsonl'
print(file_path)

/content/gdrive/My Drive/London Knowledge Prompting Hackathon/data/train.jsonl


In [ ]:
train_data = read_lm_kbc_jsonl(file_path)

In [ ]:
examples = []
for line in train_data:
  subject = line['SubjectEntity']
  relation = line['Relation']
  objects = str(line['ObjectEntities'])
  instance_dict = dict()
  instance_dict['entity_1'] = subject
  instance_dict['relation'] = relation
  instance_dict['target_entities'] = objects
  examples.append(instance_dict)

In [ ]:
# calling the embeddings
openai_embeddings = OpenAIEmbeddings()

In [ ]:
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Subject: {entity_1} => Predicate: {relation} => Object: {target_entities}
"""
example_prompt = PromptTemplate(
    input_variables=["entity_1", "relation", "target_entities"],
    template=example_formatter_template,
    )

In [ ]:
test_prompt1 = example_prompt.format(entity_1 = 'Siemens-Schuckert', relation='CompanyHasParentOrganisation', target_entities ='Siemens')
print(test_prompt1)


Subject: Siemens-Schuckert => Predicate: CompanyHasParentOrganisation => Object: Siemens



In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    openai_embeddings,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=2
    )

In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    example_selector= example_selector,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="""Generate objects holding the relation with the given subject.
    Here are some examples: """,
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="""End of the examples. Now it is your turn to generate.
    Subject: {entity_1} => Predicate: {relation} => Objects: ??? """,
    # The input variables are the variables that the overall prompt expects.
    input_variables= ["entity_1", "relation"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [ ]:
test_prompt2 = few_shot_prompt.format(entity_1 = 'Siemens-Schuckert', relation='CompanyHasParentOrganisation')
print(test_prompt2)

Generate objects holding the relation with the given subject.
    Here are some examples: 

Subject: Audi => Predicate: CompanyHasParentOrganisation => Object: ['Volkswagen Group']


Subject: Hanomag => Predicate: CompanyHasParentOrganisation => Object: ['Komatsu Limited']

End of the examples. Now it is your turn to generate.
    Subject: Siemens-Schuckert => Predicate: CompanyHasParentOrganisation => Objects: ??? 


In [ ]:
openai_llm = OpenAI(model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
openai_llm_chain = LLMChain(prompt=few_shot_prompt, llm=openai_llm, verbose=False)

In [ ]:
extraction = openai_llm_chain.run(entity_1 = 'Siemens-Schuckert', relation='CompanyHasParentOrganisation')
print(extraction)

['Siemens AG']


In [ ]:
val_data = read_lm_kbc_jsonl(Working_Dir + '/data/val.jsonl')
print(len(val_data))

1940


In [ ]:
extraction_output = []
for line in val_data:
  input_sbj = line['SubjectEntity']
  input_relation = line['Relation']
  extraction = openai_llm_chain.run(entity_1 = input_sbj, relation= input_relation)
  extraction_output.append(extraction)

In [ ]:
print(len(extraction_output))

In [ ]:
for gen in extraction_output[:10]:
  print(gen)

In [ ]:
with open(f'/content/gdrive/My Drive/London Knowledge Prompting Hackathon/generated_output/validation_results_LLM_ChatGPT.txt', 'w') as outfile:
  for i in extraction_output:
    outfile.write(f'Generated objects: {i}\n')


In [ ]:
from google.colab import files
files.download('/content/gdrive/My Drive/London Knowledge Prompting Hackathon/generated_output/validation_results_LLM_ChatGPT.txt')